# NHIS codebook import
Python script to scrape codebooks (PDF) from different years and merge them. Output is CSV

In [4]:
pip install tabula-py

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.0 MB 640.0 kB/s eta 0:00:19
   ---------------------------------------- 0.1/12.0 MB 656.4 kB/s eta 0:00:19
    --------------------------------------- 0.2/12.0 MB 1.5 MB/s eta 0:00:09
   - -------------------------------------- 0.5/12.0 MB 3.0 MB/s eta 0:00:04
   --- ------------------------------------ 1.1/12.0 MB 5.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.7/12.0 MB 6.6 MB/s eta 0:00:02
   ------- -------------------------------- 2.3/12.0 MB 7.4 MB/s eta 0:00:02
   --------- ------------------------------ 2.9/12.0 MB 8.2 MB/s eta 0:00:02
   ----------- ---------------------------- 3.6/12.0 MB 9.1 MB/s eta 0:00:01
   -------------- ------------------------- 4.4/12.0 MB 10.0 MB/s eta 0:00:01
   ------------------ --------------------- 5.4/12.0 MB 10.8 MB/s eta 0:00:01

In [5]:
pip install jpype1

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/351.5 kB ? eta -:--:--
   --- ----------------------------------- 30.7/351.5 kB 640.0 kB/s eta 0:00:01
   --------- ----------------------------- 81.9/351.5 kB 907.3 kB/s eta 0:00:01
   ------------------------------- -------- 276.5/351.5 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 351.5/351.5 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tabula
import pandas as pd

In [72]:
# List of codebooks and their path, currently grouped by child and adult. 
codebook_collections = {
    'source/child-codebook': {
        '22': 'source/child-summary22.pdf',
        '21': 'source/child-summary21.pdf',
        '20': 'source/child-summary20.pdf',
        '19': 'source/child-summary19.pdf',
    },
    'source/adult-codebook': {
        '22': 'source/adult-summary22.pdf',
        '21': 'source/adult-summary21.pdf',
        '20': 'source/adult-summary20.pdf',
        '19': 'source/adult-summary19.pdf',

    }
}

#List of columns in the PDF
data_columns = [ "Variable",
            "Question #",
            "Variable Name",
            "Source Variables",
            "Description",
            "Type",
            "Location",
            "Length"]

col_mapping = dict(zip(range(0,8), data_columns))


for cc_index, cc_value in codebook_collections.items():
    export_table = pd.DataFrame(columns=data_columns)
    for codebook_index, codebook in cc_value.items():
        base_table = pd.DataFrame(columns=range(0,7))
        
        #Import/scrape PDF, follwed by iterating through the tables in the PDF into one dataframe.
        pdf_tables = tabula.read_pdf(codebook, pages='all', multiple_tables=True, pandas_options={'header':'None'})
        for i, table in enumerate(pdf_tables):
            base_table = pd.concat([base_table, table], ignore_index=True, axis=0)
        
        base_table = base_table.rename(columns=col_mapping)
        base_table.drop_duplicates(inplace=True)
        base_table.drop(axis=0, index=0,inplace=True)
        base_table[codebook_index] = 'x'
        #There are some garbage imports (in 2019). Cleaning them up
        base_table = base_table.dropna(subset=['Type'],axis=0)
        
        #For initial PDF, just concat and don't do anything complicated
        if export_table.shape[0] == 0:
            export_table = pd.concat([export_table,base_table],axis=0)
        #For subsequent PDFs, first merge with an outer join, followed by dropping duplicates (using variable name), and then merge the description columns into one
        else:
            export_table = pd.merge(export_table,base_table,on='Variable Name',how='outer',suffixes=('', '_x'))
            export_table.drop_duplicates(subset=['Variable Name'],inplace=True,keep='first')
            for column in data_columns:
                if column != "Variable Name":
                    export_table[column] = export_table[column].combine_first(export_table[column + '_x'])
                    export_table = export_table.drop(column+'_x',axis=1)
    export_table.to_csv(cc_index +".csv")
           

In [73]:
export_table

,Variable,Question #,Variable Name,Source Variables,Description,Type,Location,Length,22,21,20,19
0,1,NaN,RECTYPE,NaN,Record type,Num,1 - 2,2,x,x,x,x
1,2,NaN,SRVY_YR,NaN,Year of the National Health Interview Survey,Num,3 - 6,4,x,x,x,x
2,3,NaN,HHX,NaN,Randomly assigned household number unique to a...,Char,7 - 13,7,x,x,x,x
3,4,NaN,WTFA_A,NaN,Weight - Final Annual,Num,14 - 23,10,x,x,x,x
4,1,Recode,URBRRL,NaN,2013 NCHS Urban-Rural Classification Scheme fo...,Num,24,1,x,x,x,x
...,...,...,...,...,...,...,...,...,...,...,...,...
957,2.0,ART.0020.00.3,JNTPN_A,NaN,"Arthritis pain, past 30 days",Num,530 - 531,2,NaN,NaN,NaN,x
958,3.0,ART.0030.00.3,ARTHLMT_A,NaN,Arthritis activity limitations,Num,532,1,NaN,NaN,NaN,x
959,4.0,ART.0040.00.3,ARTHWRK_A,NaN,Arthritis work limitations,Num,533,1,NaN,NaN,NaN,x
960,5.0,ART.0050.00.3,ARTHWT_A,NaN,Lose weight to help with arthritis,Num,534,1,NaN,NaN,NaN,x


In [3]:
pdf_table = tabula.read_pdf('source/adult-codebook19.pdf', pages='all', multiple_tables=True, pandas_options={'header':'None'})
 

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.